In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import time
import random
from abc import ABC, abstractmethod

# --- ส่วนที่ 3: Advanced OOP & Magic Methods ---

class Worker(ABC):
    @abstractmethod
    def execute(self):
        pass

class Task(Worker):
    def __init__(self, task_id, name, duration):
        self.task_id = task_id
        self.name = name
        self.duration = duration
        self.progress = 0
        self.status = "Pending"

    def execute(self, progress_callback):
        """จำลองการทำงาน (Simulation)"""
        self.status = "Running"
        step_time = self.duration / 100
        for i in range(101):
            time.sleep(step_time) # จำลองความหน่วง
            self.progress = i
            # Callback กลับไปที่ GUI เพื่ออัปเดต
            if progress_callback:
                progress_callback(self.task_id, self.progress)
        self.status = "Completed"

    # Magic Method: __str__ สำหรับการแสดงผล text
    def __str__(self):
        return f"Task[{self.task_id}]: {self.name} ({self.duration}s)"

    # Magic Method: __eq__ สำหรับการเปรียบเทียบ
    def __eq__(self, other):
        if isinstance(other, Task):
            return self.task_id == other.task_id
        return False

# --- ส่วนที่ 1 & 2: GUI & Concurrency ---

class TaskManagerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Concurrent Task Manager")
        self.root.geometry("500x400")
        
        self.tasks = []
        self.task_widgets = {} # เก็บ Widget อ้างอิงตาม Task ID

        # UI Setup
        self.setup_ui()

    def setup_ui(self):
        # Control Panel
        control_frame = ttk.Frame(self.root, padding=10)
        control_frame.pack(fill=tk.X)

        self.btn_add = ttk.Button(control_frame, text="Add Task", command=self.add_task)
        self.btn_add.pack(side=tk.LEFT, padx=5)

        self.btn_start = ttk.Button(control_frame, text="Start All", command=self.start_all_tasks)
        self.btn_start.pack(side=tk.LEFT, padx=5)

        # Task List Area (Scrollable)
        self.canvas = tk.Canvas(self.root)
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

    def add_task(self):
        t_id = len(self.tasks) + 1
        duration = random.randint(2, 5) # สุ่มเวลา 2-5 วินาที
        new_task = Task(t_id, f"Download File #{t_id}", duration)
        
        # ป้องกัน Task ซ้ำ (ทดสอบ __eq__)
        if new_task in self.tasks:
            print("Duplicate task detected!")
            return

        self.tasks.append(new_task)
        
        # สร้าง Widget สำหรับ Task นี้
        frame = ttk.Frame(self.scrollable_frame, padding=5, relief=tk.RIDGE)
        frame.pack(fill=tk.X, pady=2, padx=5)
        
        lbl_name = ttk.Label(frame, text=str(new_task), width=30)
        lbl_name.pack(side=tk.LEFT)
        
        progress = ttk.Progressbar(frame, length=150, mode='determinate')
        progress.pack(side=tk.LEFT, padx=10)
        
        lbl_status = ttk.Label(frame, text="Pending", width=10)
        lbl_status.pack(side=tk.LEFT)

        # เก็บ Reference ไว้เพื่ออัปเดตทีหลัง
        self.task_widgets[t_id] = {
            'progress_bar': progress,
            'status_label': lbl_status
        }

    def update_task_ui(self, task_id, value):
        """ฟังก์ชันนี้ถูกเรียกจาก Thread อื่น ต้องระวังเรื่อง Thread Safety"""
        # ใน Tkinter การอัปเดต UI จาก Thread อื่นโดยตรงอาจไม่ปลอดภัย 100% ในงานสเกลใหญ่
        # แต่มักจะยอมรับได้ในงานสเกลเล็ก หรือควรใช้ root.after()
        # วิธีที่ปลอดภัยกว่า:
        self.root.after(0, lambda: self._update_widget(task_id, value))

    def _update_widget(self, task_id, value):
        widgets = self.task_widgets.get(task_id)
        if widgets:
            widgets['progress_bar']['value'] = value
            if value >= 100:
                widgets['status_label'].config(text="Done", foreground="green")
            else:
                widgets['status_label'].config(text=f"{value}%")

    def run_task_thread(self, task):
        # เรียก execute และส่ง callback function ไปด้วย
        task.execute(self.update_task_ui)

    def start_all_tasks(self):
        # Concurrency: Loop สร้าง Thread สำหรับทุก Task
        for task in self.tasks:
            if task.status == "Pending":
                # สร้าง Thread แยกสำหรับแต่ละงาน
                t = threading.Thread(target=self.run_task_thread, args=(task,))
                t.daemon = True # ปิดโปรแกรมแล้ว Thread ปิดด้วย
                t.start()
        
        messagebox.showinfo("System", "All tasks started successfully!")

if __name__ == "__main__":
    root = tk.Tk()
    app = TaskManagerApp(root)
    root.mainloop()